In [1]:
#Setup
%matplotlib inline
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D, AdvectionRK4, ParticleFile, ErrorCode, Variable, VectorField, Field, FieldSamplingError, plotTrajectoriesFile, GeographicPolar, Geographic, logger
from parcels import TimeConverter
import numpy as np
from glob import glob
import matplotlib
from matplotlib import pyplot as plt
from datetime import timedelta
import netCDF4 as ntc
import math
from operator import attrgetter
import time
import sys
import PIL

## Paths and files

In [2]:
path = "E:\\OceanParcels"
outpath = path + "\\data\\processed\\scenario data\\"
scenario = ["S1", "S2", "S3", "S4", "S5", "S6"]

### Canopy positions

In [6]:
ii = 43
jj = 42

# Generate and save scenarios
for sc in range(6):
    dscen = np.squeeze(
        np.genfromtxt("../../Scenarios - mac position files/indices for OP/" + scenario[sc] + ".csv", 
                      delimiter = ","))
    canopy = np.zeros(shape = (jj, ii), dtype = 'int')
    for n in range(9):
        ai = int(dscen[n,0]-1)
        bi = int(dscen[n,1])
        aj = int(dscen[n,2]-1)
        bj = int(dscen[n,3])
        ilist = range(ai, bi)
        jlist = range(aj, bj)
        for xi in range(ii):
            for yi in range(jj):
                if xi in ilist and yi in jlist:
                    canopy[yi,xi] = 1
                    
    np.save(outpath + scenario[sc] + ".npy", canopy)

### Kelp density data

In [15]:
# From 1 year of model run, starting May 1st
D = np.genfromtxt('../data/raw/year_D.csv', delimiter = ',')[:,1] # First column is date, starts at 1/5/22
nD = sum(np.shape(D))
D = D[1:nD] # One expty space at the beginning, one extra day at the end
print(sum(np.shape(D)))

np.save(outpath+"D", D)
print(D[0:5])

365
[0.00373025 0.00373025 0.0039314  0.00412141 0.00430297]


In [14]:
mons = np.array(["may", "jun", "jul", "aug", "sep", "oct", "nov", "dec", "jan", "feb", "mar", "apr"])
days = np.array([31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30])
cday = np.array([120, 151, 181, 212, 243, 273, 303, 334, 0, 31, 59, 90])
dday = np.array([0,  
                 0+days[0],  
                 0+days[0]+days[1],  
                 0+days[0]+days[1]+days[2], 
                 0+days[0]+days[1]+days[2]+days[3], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5]+days[6], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5]+days[6]+days[7], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5]+days[6]+days[7]+days[8], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5]+days[6]+days[7]+days[8]+days[9], 
                 0+days[0]+days[1]+days[2]+days[3]+days[4]+days[5]+days[6]+days[7]+days[8]+days[9]+days[10]])

for i in range(12):
    x = np.linspace(cday[i], cday[i]+days[i], days[i])
    y = D[dday[i]:(dday[i]+days[i])]
    #plt.plot(x, y, 'o')
    xvals = np.linspace(cday[i], cday[i]+days[i], days[i]*24*6) # output is in ten-minute intervals!
    yinterp = np.interp(xvals, x, y)
    #print(sum(np.shape(yinterp)))
    #plt.plot(xvals, yinterp, '-x')
    np.save(outpath+"D_interp_"+mons[i]+".npy", yinterp)

### Relative canopy height

In [18]:
# Above, with scens
print(np.nanmin(hc), np.nanmax(hc))

0.19246874666910302 0.24693287489255294


## Define attenuation function